In [2]:
import pdfplumber

The following code block will scrape all text from the parcel report PDF -- which was generated from scraping KGIS.

In [10]:
pdf = pdfplumber.open("Parcel_Report.pdf")
all_pages = pdf.pages
for each_page in all_pages[:1]:

    text_of_each_page = each_page.extract_text()
    # skip very first line
    # read the next two lines
    # skip the next two lines
    all_lines = text_of_each_page.splitlines()
    cleaned_lines = []
    for each_line in all_lines:
        if each_line == "Address":
            continue
        elif each_line == "Owner Card  Mailing Label  Map and Details Report":
            continue
        elif each_line == "KGIS Parcel Report":
            continue
        else:
            cleaned_lines.append(each_line)
    # print(cleaned_lines)
    cleaned_list_of_dicts = []
    each_dict = {}
    for index, cleaned_line in enumerate(cleaned_lines):
        if index % 2 == 0:
            # id
            # {"parcel_id": "095GP014"}
            each_dict["parcel_id"] = cleaned_line
            # {"parcel_id": "095GN008"}
        else:
            # name
            # each_dict: {"parcel_id": "095GN008", "name": "parkey"}
            each_dict["name"] = cleaned_line
            # each_dict: {"parcel_id": "095", "name": "LEE"}
            cleaned_list_of_dicts.append(each_dict)
            # each_dict = {}
    print(cleaned_list_of_dicts)


[{'parcel_id': '095BM001', 'name': 'JOHN RACHEL'}, {'parcel_id': '095BM001', 'name': 'JOHN RACHEL'}, {'parcel_id': '095BM001', 'name': 'JOHN RACHEL'}, {'parcel_id': '095BM001', 'name': 'JOHN RACHEL'}, {'parcel_id': '095BM001', 'name': 'JOHN RACHEL'}, {'parcel_id': '095BM001', 'name': 'JOHN RACHEL'}, {'parcel_id': '095BM001', 'name': 'JOHN RACHEL'}, {'parcel_id': '095BM001', 'name': 'JOHN RACHEL'}]
